In [1]:
# Import needed dependencies
import requests
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
today = date.today()

# dd/mm/YY
current_year = today.strftime("%Y")

int(current_year) - 1

2021

In [3]:
last_five_years = []
for i in range(1,6):
    last_five_years.append(int(current_year) - i)

last_five_years

[2021, 2020, 2019, 2018, 2017]

In [4]:
# Create a function to create a dataframe from Baseball Reference tables
def batting_stats_dataframe():
    
    # input URL and use BeautifulSoup to parse through the page
    url = f'https://www.baseball-reference.com/leagues/majors/{int(current_year) - 1}-standard-batting.shtml'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Grab the table element that has batter statistics
    table = BeautifulSoup(soup.select_one('#all_players_standard_batting').find_next(text=lambda x: isinstance(x, Comment)), 'html.parser')

    # Create a list to help create a dataframe from batter statistics data
    batter_stats = []

    # Grab data from table and put it into the list created above
    for tr in table.select('tr:has(td)'):
        tds = [td.get_text(strip=True) for td in tr.select('td')]
        batter_stats.append(tds)

    # Create dataframe for batter statistics
    batter_stats_df = pd.DataFrame(batter_stats)

    # Create an empty list to store dataframe header information
    header_list = []

    # Grab the table header information to use as column headers in our dataframe
    for tr in table.select('tr:has(th)'):
        ths = [th.get_text(strip=True) for th in tr.select('th')]
        header_list.append(ths)

    # For loop returns a list of lists, and we only need the first list 
    df_headers = header_list[0]

    # Remove the first item from our headers list, it is the index header that we do not need
    df_headers.remove('Rk')

    # Set column headers equal to our list
    batter_stats_df.columns = df_headers

    # return the dataframe
    return batter_stats_df

# # use a for loop to call the function for the past five years' worth of batting stats data
# batting_stats_dataframe()


In [5]:
batting_stats_dataframe()

,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary
0,Fernando Abad*,35,BAL,AL,2,0,0,0,0,0,...,,,,0,0,0,0,0,0,1
1,Cory Abbott,25,CHC,NL,8,3,3,0,1,0,...,.333,.667,81,1,0,0,0,0,0,/1H
2,Albert Abreu,25,NYY,AL,3,0,0,0,0,0,...,,,,0,0,0,0,0,0,1
3,Bryan Abreu,24,HOU,AL,1,0,0,0,0,0,...,,,,0,0,0,0,0,0,1
4,José Abreu,34,CHW,AL,152,659,566,86,148,30,...,.481,.831,125,272,28,22,0,10,3,*3D/5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1719,Bruce Zimmermann*,26,BAL,AL,2,4,4,0,0,0,...,.000,.000,-100,0,0,0,0,0,0,1
1720,Jordan Zimmermann,35,MIL,NL,2,1,1,0,0,0,...,.000,.000,-100,0,0,0,0,0,0,/1
1721,Tyler Zuber,26,KCR,AL,1,1,1,0,0,0,...,.000,.000,-100,0,0,0,0,0,0,1
1722,Mike Zunino,30,TBR,AL,109,375,333,64,72,11,...,.559,.860,138,186,7,7,0,1,0,2/H
